[Slides](./Networking.slides.html)

# Networking

Python has support for both high-level network protocols (HTTP/HTTPS/FTP/SMTP) as well as low-level protocols (sockets).

In this module you'll learn

- How to use `urllib`/`urllib2` to fetch web resources
- How to send email using the `smtplib` and `email` modules
- How to do basic socket programming in Python
- How you might build a REST API client in Python

## Fetching web resources with urllib/urllib2

The `urllib` and `urllib2` standard library modules allow you to open a URL (almost) just like a file. For example:

In [14]:
import urllib
import urllib.request as urllib2
from urllib.parse import urlencode

In [9]:
resp = urllib2.urlopen('http://www.bloomberg.net')
html = resp.read()
len(html)

69868

You can also POST data to a URL using a `Request` and get the response:

In [17]:
url = 'http://localhost:8000/cgi-bin/print_request.py'
values = {'name' : 'Rick Copeland',
          'state' : 'Georgia' }

data = urlencode(values).encode('utf-8')
req = urllib2.Request(url, data)
response = urllib2.urlopen(req)
print(response.read())

b'<TITLE>CGI script output</TITLE>\n<h1>What you submitted</h1>\n<table><tbody>\n<tr><th>state</th><td>Georgia</td></tr>\n<tr><th>name</th><td>Rick Copeland</td></tr>\n</tbody></table>\n<a href="/">Return to index</a>\n'


In [19]:
print(data)

b'name=Rick+Copeland&state=Georgia'


In [22]:
url = 'http://localhost:8000/cgi-bin/print_json.py'
values = {'name' : 'Rick Copeland',
          'state' : 'Georgia' }

data = urlencode(values).encode('utf-8')
req = urllib2.Request(url, data)
response = urllib2.urlopen(req)
print(response.read())

b'{"state": "Georgia", "HTTP_METHOD": "POST", "name": "Rick Copeland"}\n'


You can also pass query parameters to a request using `urlencode`:

In [27]:
encoded_values = urlencode(values)
encoded_values

'name=Rick+Copeland&state=Georgia'

In [29]:
print(urllib2.urlopen(url + '?' + encoded_values).read())

b'{"state": "Georgia", "HTTP_METHOD": "GET", "name": "Rick Copeland"}\n'


## Sending email with `smtplib`

It's frequently useful to send emails from long-running scripts; Python makes this easy with `smtplib`:

In [30]:
import smtplib

Most of the time, you'll be connecting to either your ISP's SMTP server, or some other server with authentication. In order to avoid exposing my password and such, I'll just use a local proxy server to print some debugging information:

(You can run the proxy with `python -m smtpd -nd`)

In [31]:
conn = smtplib.SMTP('localhost', 8025)
conn.set_debuglevel(1)
# conn.login(username, password)
addr_from = 'rick@arborian.com'
addr_to = 'rick446@usa.net'
msg = '''Subject: This is a test
From: <{}>
To: <{}>

Test message
'''.format(addr_from, addr_to)
conn.sendmail(addr_from, [addr_to], msg)

send: 'ehlo 81.1.168.192.in-addr.arpa\r\n'
reply: b'502 Error: command "EHLO" not implemented\r\n'
reply: retcode (502); Msg: b'Error: command "EHLO" not implemented'
send: 'helo 81.1.168.192.in-addr.arpa\r\n'
reply: b'250 81.1.168.192.in-addr.arpa\r\n'
reply: retcode (250); Msg: b'81.1.168.192.in-addr.arpa'
send: 'mail FROM:<rick@arborian.com>\r\n'
reply: b'250 Ok\r\n'
reply: retcode (250); Msg: b'Ok'
send: 'rcpt TO:<rick446@usa.net>\r\n'
reply: b'250 Ok\r\n'
reply: retcode (250); Msg: b'Ok'
send: 'data\r\n'
reply: b'354 End data with <CR><LF>.<CR><LF>\r\n'
reply: retcode (354); Msg: b'End data with <CR><LF>.<CR><LF>'
data: (354, b'End data with <CR><LF>.<CR><LF>')
send: b'Subject: This is a test\r\nFrom: <rick@arborian.com>\r\nTo: <rick446@usa.net>\r\n\r\nTest message\r\n.\r\n'
reply: b'250 Ok\r\n'
reply: retcode (250); Msg: b'Ok'
data: (250, b'Ok')


{}

If you're going to be putting together more complicated emails, the `email` module is extremely useful (for multipart messages, attachments, etc.):

In [32]:
from email.mime.text import MIMEText

In [33]:
msg = MIMEText('This is a text message')
msg['Subject'] = 'This is a test message'
msg['From'] = addr_from
msg['To'] = addr_to
conn.sendmail(addr_from, [addr_to], msg.as_string())

send: 'mail FROM:<rick@arborian.com>\r\n'
reply: b'250 Ok\r\n'
reply: retcode (250); Msg: b'Ok'
send: 'rcpt TO:<rick446@usa.net>\r\n'
reply: b'250 Ok\r\n'
reply: retcode (250); Msg: b'Ok'
send: 'data\r\n'
reply: b'354 End data with <CR><LF>.<CR><LF>\r\n'
reply: retcode (354); Msg: b'End data with <CR><LF>.<CR><LF>'
data: (354, b'End data with <CR><LF>.<CR><LF>')
send: b'Content-Type: text/plain; charset="us-ascii"\r\nMIME-Version: 1.0\r\nContent-Transfer-Encoding: 7bit\r\nSubject: This is a test message\r\nFrom: rick@arborian.com\r\nTo: rick446@usa.net\r\n\r\nThis is a text message\r\n.\r\n'
reply: b'250 Ok\r\n'
reply: retcode (250); Msg: b'Ok'
data: (250, b'Ok')


{}

In [34]:
conn.quit()

send: 'quit\r\n'
reply: b'221 Bye\r\n'
reply: retcode (221); Msg: b'Bye'


(221, b'Bye')

# Socket programmming

If you want to use a protocol that Python doesn't support natively (or just want to use your own protocol), you can always use the lower-level `socket` module in the standard library:

In [35]:
import socket

In [40]:
# sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock = socket.socket()

In [41]:
sock.connect(('www.bloomberg.com', 80))
http_req = b'''GET / HTTP/1.1
Host: www.bloomberg.com
User-Agent: Intermediate-Python/1.0
Accept: */*

'''
sock.sendall(http_req)
response = sock.recv(1024)
sock.close()
print(len(response))

766


In [42]:
print(response)

b"HTTP/1.1 301 Moved Permanently\r\nServer: Apache\r\nContent-Security-Policy: upgrade-insecure-requests\r\nX-Content-Security-Policy: upgrade-insecure-requests\r\nLocation: https://www.bloomberg.com/\r\nContent-Type: text/html; charset=iso-8859-1\r\nVary: Accept-Encoding\r\nContent-Encoding: gzip\r\nContent-Length: 193\r\nCache-Control: max-age=68\r\nExpires: Thu, 16 Mar 2017 03:14:50 GMT\r\nDate: Thu, 16 Mar 2017 03:13:42 GMT\r\nConnection: keep-alive\r\nSet-Cookie: ak_rg=US; expires=Thu, 23-Mar-2017 03:13:42 GMT; path=/\r\nSet-Cookie: ak_co=US; expires=Thu, 23-Mar-2017 03:13:42 GMT; path=/\r\n\r\n\x1f\x8b\x08\x00\x00\x00\x00\x00\x00\x03m\x8e\xb1\x0e\x82@\x10D{\xbeb\xa5\x97\x05\xed\xccz\x85\x80\xd1\x04\x94\xe2,,\x0fX=\x13\x8e#xJ\xfc{\x01[\xcb\x99y;\xb3\xb4H\xce\xb1\xbc\x16)\x1cd\x9eAq\xd9e\xc7\x18\xfc%\xe21\x95{\xc4D&\xbfd\x15\x84\x88\xe9\xc9\x17\x1eig\x1aA\x9aU=\n\xf7p\r\x8bu\x18An\xdf\\C\xc1\xbdQ-\xb7\xae\xf9\x10\xfeB\x8fp\x86\xa9\xb4\xf5g\xba\x8f\xc4\x1fvt=\xea\x84\xd4\x0c\xb5\xad

You can, of course, also build a socket server:

In [43]:
sock_srv = socket.socket()
sock_srv.bind(('127.0.0.1', 8042))
sock_srv.listen(1)
sock_conn, cli_addr = sock_srv.accept()
print('Got a connection from', cli_addr)
msg = sock_conn.recv(1024)
print(msg)
sock_conn.sendall(msg)
sock_conn.close()

Got a connection from ('127.0.0.1', 52315)
b'Hello\n'


Normally, you'd do something more like the following:

In [45]:
import threading

def handle_echo_connection(sock, addr):
    print('Handling connection from', addr)
    while True:
        msg = sock.recv(1024)
        if not msg:
            break
        print('Got', msg.rstrip())
        sock.sendall(msg)
    sock.close()
    print('Exiting handler')
        
sock_srv = socket.socket()
sock_srv.bind(('127.0.0.1', 8043))
sock_srv.listen(1)

while True:
    sock_conn, cli_addr = sock_srv.accept()
    thd = threading.Thread(target=handle_echo_connection, args=(sock_conn, cli_addr))
    thd.setDaemon(True)
    thd.start()
    break   # normally we'd keep looping

print('Closing server socket')
sock_srv.close()

Handling connection fromClosing server socket
 ('127.0.0.1', 52322)
Got b'Hello'
Got b'there'
Exiting handler


## Building a simple JSON REST API client

We're going to use a simple in-memory key/value store with a few endpoints:

 - GET / - return a list of collection names
 - GET /{cname} - return a list of ids for documents in a collection
 - POST /{cname} - insert a document (with auto-generated id) in a collection
 - DELETE /{cname} - drop a collection
 - GET /{cname}/{id} - return a document
 - PUT /{cname}/{id} - insert/replace a document
 - DELETE /{cname}/{id} - delete a document

In [47]:
import json

class Database(object):
    
    def __init__(self, url):
        self._url = url
        
    def collections(self):
        resp = json.load(urllib2.urlopen(self._url))
        return resp

In [49]:
db = Database('http://localhost:5000')
db.collections()

{'collections': []}

Unfortunately, `urllib2` requests are a little broken when it comes to HTTP methods, so we'll have to subclass them:

In [50]:
class PUT(urllib2.Request):
    def get_method(self):
        return 'PUT'

class DELETE(urllib2.Request):
    def get_method(self):
        return 'DELETE'

In [57]:
class Database(object):
    
    def __init__(self, url):
        self._url = url
        
    def collections(self):
        resp = json.load(urllib2.urlopen(self._url))
        return resp
    
    def _href_collection(self, cname):
        return '%s/%s' % (self._url, cname)
    
    def __getitem__(self, cname):
        return Collection(self._href_collection(cname))
    
    def __delitem__(self, cname):
        req = DELETE(self._href_collection(cname))
        return json.load(urllib2.urlopen(req))

class Collection(object):
    
    def __init__(self, url):
        self._url = url
        
    def documents(self):
        resp = json.load(urllib2.urlopen(self._url))
        return resp
    
    def drop(self):
        req = DELETE(self._url)
        return json.load(urllib2.urlopen(req))
    
    def insert(self, doc):
        return json.load(urllib2.urlopen(
            self._url, json.dumps(doc).encode('utf-8')))
    
    def _href_doc(self, id):
        return '%s/%s' % (self._url, id)
    
    def __getitem__(self, id):
        return json.load(urllib2.urlopen(self._href_doc(id)))
    
    def __setitem__(self, id, value):
        req = PUT(
            self._href_doc(id), 
            json.dumps(value).encode('utf-8'), 
            {'Content-Type': 'application/json'})
        return json.load(urllib2.urlopen(req))
    
    def __delitem__(self, id):
        req = DELETE(self._href_doc(id))
        response = urllib2.urlopen(req)
        return json.load(response)
    
    

In [58]:
db = Database('http://localhost:5000')
coll = db['foo']
coll['0'] = {'name': 'Foo'}
db.collections()

{'collections': [{'cname': 'foo', 'href': '/foo'}]}

In [59]:
coll.documents()

{'docs': [{'href': '/foo/0', 'id': '0'}]}

In [60]:
coll.insert({'name': 'Bar'})

{'href': '/foo/K81Ai6RnAqGshf5K', 'id': 'K81Ai6RnAqGshf5K'}

In [61]:
docs = coll.documents()
docs = dict((d['id'], d['href']) for d in docs['docs'])
docs

{'0': '/foo/0', 'K81Ai6RnAqGshf5K': '/foo/K81Ai6RnAqGshf5K'}

In [62]:
coll[0]

{'name': 'Foo'}

In [63]:
del db['foo']['0']

In [64]:
coll.documents()

{'docs': [{'href': '/foo/K81Ai6RnAqGshf5K', 'id': 'K81Ai6RnAqGshf5K'}]}

In [65]:
coll.drop()

{}

In [66]:
db.collections()

{'collections': []}

Python Requests library (3rd party) simplifies many of these tasks.

http://docs.python-requests.org/en/master/